In [1]:
# Challenge Sorbonne - DST
#
#!pip install torch_geometric

import numpy as np
import pandas as pd
import os
import time
import os
import json
#import rarfile
import networkx as nx
import io
import re
import random
import csv
from datetime import datetime

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import random_split, Subset
from torch.optim import AdamW
#!pip install torch_geometric
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GATConv, global_mean_pool
import networkx as nx

from collections import defaultdict, Counter

import matplotlib.pyplot as plt
import plotly.io as pio
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from sklearn.preprocessing import LabelEncoder, RobustScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import tqdm as notebook_tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device is", device)

current_directory = os.getcwd()
print(f"The current working directory is: {current_directory}")

#env_system = os.environ
#print(env_system)

KeyboardInterrupt: 

In [ ]:
# for Google Colab only
if 'GOOGLE_COLAB' in os.environ:
    from google.colab import drive
    drive.mount('/content/drive')

In [ ]:
current_directory = os.getcwd()
print(f"The current working directory is: {current_directory}")

# init du repertoire des données
if current_directory == '/usr/users/msiac/msiac_10/Sorbonne' : #DCE
    training_path_dir =   "./training"
    test_path_dir = "./test"

    train_meta_data = 'training_set_metadata.csv'
    model_save_file =  'model_sorbonne_weights.pth'

    split_char = '/'
    filename_test_results = "test_prediction.csv"
    filename_trained = "list_hash_trained.csv"
    file_split_training = 'split_training-dce.csv'

elif current_directory == '/content': # Google Colab
    training_path_dir = "/content/drive/MyDrive/Sorbonne_Data_challenge/folder_training_set"
    test_path_dir = "/content/drive/MyDrive/Sorbonne_Data_challenge/folder_test_set"

    train_meta_data =  '/content/drive/MyDrive/Sorbonne_Data_challenge/training_set_metadata.csv'

    model_save_file =  '/content/drive/MyDrive/Colab Notebooks/Sorbonne-Challenge/model_sorbonne_weights.pth'
    file_split_training = '/content/drive/MyDrive/Colab Notebooks/Sorbonne-Challenge/split_training-colab.csv'
    split_char = '/'

    filename_test_results = '/content/drive/MyDrive/Colab Notebooks/Sorbonne-Challenge/test_prediction.csv'
    filename_trained = '/content/drive/MyDrive/Colab Notebooks/Sorbonne-Challenge/list_hash_trained.csv'

elif current_directory == r"c:\Users\jch_m\DocumentsPerso\Centrale-IAC\Cours\Python-ML\ProjectMaster\Challenge-Sorbonne": #PC1
    split_char = '\\'
    
    #training_path_dir =   r"D:\ChallengeDST\folder_training_set"
    training_path_dir =   r"..\..\Data\ChallengeSorbonne\training"
    #training_path_dir =   r"..\..\Data\ChallengeSorbonne\training_s"
    filename_trained = r".\list_hash_trained.csv"

    train_meta_data =  r'..\..\Data\ChallengeSorbonne\training_set_metadata.csv'
    file_split_training = r'.\split_training-pc.csv'

    #test_path_dir =   r"D:\ChallengeDST\folder_training_set\folder_training_set"
    test_path_dir = r"..\..\Data\ChallengeSorbonne\training"
    test_path_dir = r"G:\Mon Drive\Colab Notebooks\Sorbonne-Challenge"
    test_init_file = r".\test_set_to_predict.csv"
    filename_test_results = r".\test_prediction.csv"
    
    model_save_file =  r'.\model_sorbonne_weights.pth'
    filename_test_results = r".\test_prediction.csv"

elif current_directory == r"c:\Users\jch_m\Documents Perso\DevPython\Challenge-sorbonne": #PC2
    split_char = '\\'
    
    #training_path_dir =   r"D:\ChallengeDST\folder_training_set"
    training_path_dir =   r"..\..\Data\ChallengeSorbonne\training"
    #training_path_dir =   r"..\..\Data\ChallengeSorbonne\training_s"
    filename_trained = r".\list_hash_trained.csv"

    train_meta_data =  'training_set_metadata.csv'
    file_split_training = r'.\split_training-pc.csv'

    #test_path_dir =   r"D:\ChallengeDST\folder_training_set\folder_training_set"
    #test_path_dir = r"..\..\Data\ChallengeSorbonne\training"
    test_path_dir = r"G:\.shortcut-targets-by-id\19Ls71oZUG1aa1uTMu1laH6Nt3qK5gSZK\Sorbonne_Data_challenge\folder_test_set"
    test_init_file = "test_set_to_predict.csv"
    filename_test_results = "test_prediction.csv"
    model_save_file =  'model_sorbonne_weights.pth'
else:
    print("**** ERROR: init files names and directories not done - root directory and environment not identified")

# read CSV input and save in df
df_meta_train = pd.read_csv(train_meta_data, sep=";")
df_meta_train.head()


The current working directory is: c:\Users\jch_m\Documents Perso\DevPython\Challenge-sorbonne


,name,64-bit execution via heavens gate,64bits,PEB access,accept command line arguments,access the Windows event log,act as TCP client,allocate RW memory,allocate RWX memory,allocate memory,...,winzip,wise,worm,write and execute a file,write clipboard data,write file on Linux,write file on Windows,write pipe,xorcrypt,yoda
0,9fbf213113ba0a18dc2642f83b1201541428fd7951d6a8...,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1b35c9dbf3cd9ac60015aaa6cd451c898defa6dac1ff43...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,bf8d307a136a936f7338c1f2eec773c4eb1c802cab77da...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1e51933903f0358c0b635f863368eb15a61cd3442bc5bf...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8a6503fe68d699f8a31531c157e9da931192cd7e3ec809...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df_meta_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23102 entries, 0 to 23101
Columns: 454 entries, name to yoda
dtypes: int64(453), object(1)
memory usage: 80.0+ MB


In [ ]:
print(filename_trained)
df_files_trained = pd.read_csv(filename_trained, sep=",")
print("taille :",len(df_files_trained))
df_files_trained.head()


.\list_hash_trained.csv
taille : 5076


,files_trained
0,files_trained
1,861f2c5f07c9e1c7d24c2e34eb47ff3129cd39a2227a25...
2,793a1bda32069f37ba201167d90e8c16c004008c4fd467...
3,819d0b70a905ae5f8bef6c47423964359c2a90a168414f...
4,892e7b1463b98d6f5b41579c6b314de3a723af2c67d88f...


In [ ]:
# load & check traing split
df_training_split = pd.read_csv(file_split_training, sep=";")
full_file_list = df_training_split[df_training_split['batch'] == 'rep_500']['orign path'].tolist()
print(df_training_split['batch'].unique()[:3])
print(full_file_list[:5])


['rep_0' 'rep_500' 'rep_1000']
['D:\\ChallengeDST\\folder_training_set\\folder_training_set\\04333849d7eb415a3986dc1afe0041723184b7f47aeeec5ffa07a0f9c222fee3.json', 'D:\\ChallengeDST\\folder_training_set\\folder_training_set\\04390cd4165bf34f62658cea0fa16116037e391a86ed59d09493c9d141f92276.json', 'D:\\ChallengeDST\\folder_training_set\\folder_training_set\\043a5b4be0a8f0500e88a2ffc91d15bf4212fa12a3f07be04654b67adc41d21c.json', 'D:\\ChallengeDST\\folder_training_set\\folder_training_set\\043cf998d7059cacaf626f443181c67f91adeb2bc7088ca918e1aee94838d23d.json', 'D:\\ChallengeDST\\folder_training_set\\folder_training_set\\043e2a8241a6eb0aa66de40e35e75dce0f1d047a6d9e4c934b482f7c588f9c61.json']


In [ ]:
def process_graph_directory(file_directory, val_size=0.1, rep_batch='rep_0'):
    """
    Scan the training directory
    get the hash name of the files, check and exclude hash file with error
    retour a list of hash for training, validation and error
    """

    list_train_hash, list_val_hash, list_err_hash  = [], [], []
    file_with_err = 0
    count_files = 0
    max_file = 0

    #file_split_training = r"C:\Users\jch_m\DocumentsPerso\Centrale-IAC\Cours\Python-ML\ProjectMaster\ChallengeSorbonne\split_training-colab.csv"
    df_files_trained = pd.read_csv(filename_trained, sep=",")
    df_training_split = pd.read_csv(file_split_training, sep=";")
    full_file_list = df_training_split[df_training_split['batch']== rep_batch]['orign path'].tolist()

    for full_path_file in full_file_list:
        # test if filename in the trained list
        hash_name = full_path_file.split('.jso')[0]
        hash_name = hash_name.split(split_char)[-1]

        # file already trained
        if hash_name in df_files_trained['files_trained'].values:
            #print("trained")
            continue

        # files not in metadata
        if hash_name not in df_meta_train['name'].values:
            list_err_hash.append(full_path_file)
            print("meta", hash_name, full_path_file)
            continue

        if os.path.exists(full_path_file):
            file_size = os.path.getsize(full_path_file)
            if file_size/1000 > 100_000: #file too big
                list_err_hash.append(full_path_file)
                print("size", file_size/1000, full_path_file)
                continue
        else:
            list_err_hash.append(full_path_file)
            print("not exist", full_path_file)
            continue

        # check if nb of files is within the max_files allowed
        if max_file != 0 and count_files > max_file:
            list_err_hash.append(full_path_file)
            print("count")
            break

        if count_files % (val_size*100) == 0:
            list_val_hash.append(full_path_file)
        else:
            list_train_hash.append(full_path_file)

        count_files += 1

    print(f"Number of files with error: {len(list_err_hash)}")
    return list_train_hash, list_val_hash, list_err_hash

In [ ]:
list_train, list_val, list_err = process_graph_directory(training_path_dir, val_size=0.1, rep_batch='rep_2500')
print(len(list_val), len(list_train), len(list_err))

not exist D:\ChallengeDST\folder_training_set\folder_training_set\16e40e92dd11003a960e6944be261ef59e988599437c9e0a025d94a315abfe55.json
not exist D:\ChallengeDST\folder_training_set\folder_training_set\16ff5ba3a1cd45b8886f10f61cefa4140c09be357240a1b8233c94f8bd6758a1.json
not exist D:\ChallengeDST\folder_training_set\folder_training_set\172363c4c7b1688c62a6b0ba798dc7c5a80b119f5b9ffff7bae009a28cd3e6c8.json
not exist D:\ChallengeDST\folder_training_set\folder_training_set\1730968b5903215d77bd1f2e114f3b9e9a6cbff9bd8d25ada46d2af663422e81.json
not exist D:\ChallengeDST\folder_training_set\folder_training_set\1745d056b69ad2dd90822c062c6bfe67f2d2dee1abd3bc5c6ace0298d677244d.json
not exist D:\ChallengeDST\folder_training_set\folder_training_set\175489d3698f84221b308bfed110ea40fb5d002819d888dc94fe932239fc5b34.json
not exist D:\ChallengeDST\folder_training_set\folder_training_set\176cd962682da7c22b2dda004637798dea9c36acc12584898cfb7b645f73a2bf.json
not exist D:\ChallengeDST\folder_training_set\fo

In [ ]:
class HierarchicalAssemblyTokenizer:
    def __init__(self):
        # Tokenizer implementation from previous code
        # Abbreviated for brevity but would be the full implementation
        self.node_type_vocab = {}
        self.operation_vocab = {}
        self.register_vocab = {}
        self.memory_pattern_vocab = {}
        self.immediate_vocab = {}
        self.UNK_TOKEN = "<UNK>"

        # Regex patterns for parsing
        self.patterns = {
            'node_type': re.compile(r'([A-Z]+)\s*:'),
            'operation': re.compile(r':\s*([a-z]+)'),
            'registers': re.compile(r'(?:^|\s+)([a-z]{2,3}x|[a-z]{2,3}i|[a-z]{1,3}h|[a-z]{1,3}l)(?:$|\s+|,|\])'),
            'memory_ref': re.compile(r'\[(.*?)\]'),
            'immediate': re.compile(r'0x[0-9a-fA-F]+|\d+')
        }

    def fit(self, graph_l, min_freq=1):
        """Build vocabularies from the digraph nodes"""
        node_types = []
        operations = []
        registers = []
        memory_patterns = []
        immediates = []
        self.unique_hashes = []
        self.hash_to_id = {}
        self.id_to_hash = {}

        # Extract features from node labels
        for item in graph_l:
            graph = item['graph_input']
            for node_id, node_attr in graph.nodes(data=True):
                label = node_attr.get('label', '')

                # Extract node type (JCC, INST)
                node_type_match = self.patterns['node_type'].search(label)
                if node_type_match:
                    node_types.append(node_type_match.group(1))

                # Extract operation (xor, push, mov)
                op_match = self.patterns['operation'].search(label)
                if op_match:
                    operations.append(op_match.group(1))

                # Extract registers
                reg_matches = self.patterns['registers'].findall(label)
                registers.extend(reg_matches)

                # Extract memory reference patterns
                mem_matches = self.patterns['memory_ref'].findall(label)
                for mem in mem_matches:
                    # Convert memory reference to a pattern (e.g., "reg + offset")
                    pattern = self._memory_to_pattern(mem)
                    memory_patterns.append(pattern)

                # Extract immediate values
                imm_matches = self.patterns['immediate'].findall(label)
                immediates.extend(imm_matches)

            self.unique_hashes.append(item['name'])
            self.unique_hashes = sorted(list(self.unique_hashes))
            for i, hash_val in enumerate(self.unique_hashes):
                self.hash_to_id[hash_val] = i
                self.id_to_hash[i] = hash_val

        # Build vocabularies with frequency filtering
        self._build_vocab(self.node_type_vocab, node_types, min_freq)
        self._build_vocab(self.operation_vocab, operations, min_freq)
        self._build_vocab(self.register_vocab, registers, min_freq)
        self._build_vocab(self.memory_pattern_vocab, memory_patterns, min_freq)
        self._build_vocab(self.immediate_vocab, immediates, min_freq)

        return self

    def fit_from_counts(self, all_counts, min_freq=3):
        """Fits the tokenizer from accumulated counts."""
        self.node_type_vocab.clear()
        self.operation_vocab.clear()
        self.register_vocab.clear()
        self.memory_pattern_vocab.clear()
        self.immediate_vocab.clear()

        self._build_vocab(self.node_type_vocab, all_counts['node_types'], min_freq)
        self._build_vocab(self.operation_vocab, all_counts['operations'], min_freq*2)
        self._build_vocab(self.register_vocab, all_counts['registers'], min_freq)
        self._build_vocab(self.memory_pattern_vocab, all_counts['memory_patterns'], min_freq*3)
        self._build_vocab(self.immediate_vocab, all_counts['immediates'], min_freq*3)

        self.unique_hashes = all_counts['unique_hashes']
        self.unique_hashes = sorted(list(self.unique_hashes))
        for i, hash_val in enumerate(self.unique_hashes):
            self.hash_to_id[hash_val] = i
            self.id_to_hash[i] = hash_val

    def _build_vocab(self, vocab_dict, tokens, min_freq):
        """Build vocabulary with frequency filtering"""
        counter = Counter(tokens)
        # Add special UNK token
        vocab_dict[self.UNK_TOKEN] = 0

        # Add tokens that meet minimum frequency
        idx = 1
        for token, count in counter.most_common():
            if count >= min_freq:
                vocab_dict[token] = idx
                idx += 1

    def _memory_to_pattern(self, mem_ref):
        """Convert memory reference to pattern"""
        # Replace registers with REG placeholder
        pattern = re.sub(r'[a-z]{2,3}x|[a-z]{2,3}i|[a-z]{1,3}h|[a-z]{1,3}l', 'REG', mem_ref)
        # Replace immediate values with IMM placeholder
        pattern = re.sub(r'0x[0-9a-fA-F]+|\d+', 'IMM', pattern)
        return pattern.strip()

    def tokenize(self, node_label):
        """Tokenize a node label into hierarchical features"""
        features = {
            'node_type': self.UNK_TOKEN,
            'operation': self.UNK_TOKEN,
            'registers': [],
            'memory_pattern': self.UNK_TOKEN,
            'immediate': self.UNK_TOKEN
        }

        # Extract node type
        node_type_match = self.patterns['node_type'].search(node_label)
        if node_type_match:
            node_type = node_type_match.group(1)
            features['node_type'] = node_type if node_type in self.node_type_vocab else self.UNK_TOKEN

        # Extract operation
        op_match = self.patterns['operation'].search(node_label)
        if op_match:
            operation = op_match.group(1)
            features['operation'] = operation if operation in self.operation_vocab else self.UNK_TOKEN

        # Extract registers
        reg_matches = self.patterns['registers'].findall(node_label)
        features['registers'] = [reg if reg in self.register_vocab else self.UNK_TOKEN for reg in reg_matches]

        # Extract memory reference
        mem_matches = self.patterns['memory_ref'].findall(node_label)
        if mem_matches:
            pattern = self._memory_to_pattern(mem_matches[0])
            features['memory_pattern'] = pattern if pattern in self.memory_pattern_vocab else self.UNK_TOKEN

        # Extract immediate values
        imm_matches = self.patterns['immediate'].findall(node_label)
        if imm_matches:
            imm = imm_matches[0]
            features['immediate'] = imm if imm in self.immediate_vocab else self.UNK_TOKEN

        return features

    def encode_nodelabel(self, node_label):
        """Convert a node label to numeric feature vectors"""
        features = self.tokenize(node_label)

        # Encode each feature
        node_type_idx = self.node_type_vocab.get(features['node_type'], self.node_type_vocab[self.UNK_TOKEN])
        operation_idx = self.operation_vocab.get(features['operation'], self.operation_vocab[self.UNK_TOKEN])

        # Encode registers (take up to 3, pad if fewer)
        register_indices = []
        for i in range(min(3, len(features['registers']))):
            reg = features['registers'][i]
            reg_idx = self.register_vocab.get(reg, self.register_vocab[self.UNK_TOKEN])
            register_indices.append(reg_idx)

        # Pad register indices if needed
        while len(register_indices) < 3:
            register_indices.append(0)  # 0 for padding

        memory_pattern_idx = self.memory_pattern_vocab.get(
            features['memory_pattern'],
            self.memory_pattern_vocab[self.UNK_TOKEN]
        )

        immediate_idx = self.immediate_vocab.get(
            features['immediate'],
            self.immediate_vocab[self.UNK_TOKEN]
        )

        # Combine all indices into a feature vector
        encoded = np.array([
            node_type_idx,
            operation_idx,
            register_indices[0],
            register_indices[1],
            register_indices[2],
            memory_pattern_idx,
            immediate_idx
        ], dtype=np.int64)

        return encoded

    def encode_graph(self, digraph):
        """Convert an entire digraph to node feature vectors"""
        node_features = {}

        for node_id in digraph.nodes():
            label = digraph.nodes[node_id].get('label', '')
            node_features[node_id] = self.encode_nodelabel(label)

        return node_features

    def encode_hash(self, hash_file):
        return self.hash_to_id.get(hash_file, -1) # Return -1 if hash is not found

    def get_vocab_sizes(self):
        """Return the size of each vocabulary"""
        return {
            'node_type': len(self.node_type_vocab),
            'operation': len(self.operation_vocab),
            'register': len(self.register_vocab),
            'memory_pattern': len(self.memory_pattern_vocab),
            'immediate': len(self.immediate_vocab)
        }

    def get_tokens_and_counts(self, graph_l):
        """Extracts tokens and counts from graph_l."""
        node_types = []
        operations = []
        registers = []
        memory_patterns = []
        immediates = []
        unique_hashes = []
        self.hash_to_id = {}
        self.id_to_hash = {}

        for item in graph_l:
            graph = item['graph_input']
            for node_id, node_attr in graph.nodes(data=True):
                label = node_attr.get('label', '')

                node_type_match = self.patterns['node_type'].search(label)
                if node_type_match:
                    node_types.append(node_type_match.group(1))

                op_match = self.patterns['operation'].search(label)
                if op_match:
                    operations.append(op_match.group(1))

                reg_matches = self.patterns['registers'].findall(label)
                registers.extend(reg_matches)

                mem_matches = self.patterns['memory_ref'].findall(label)
                for mem in mem_matches:
                    pattern = self._memory_to_pattern(mem)
                    memory_patterns.append(pattern)

                imm_matches = self.patterns['immediate'].findall(label)
                immediates.extend(imm_matches)

            unique_hashes.append(item['name'])

        return {
            'node_types': node_types,
            'operations': operations,
            'registers': registers,
            'memory_patterns': memory_patterns,
            'immediates': immediates,
            'unique_hashes': unique_hashes,
        }

class AssemblyGraphDataset:
    def __init__(self, graph_list, tokenizer, node_feature_dim=60):
        """
        Prepares a dataset of assembly graphs for GAT model training
        Args:
            graph_list: List of NetworkX digraphs representing assembly code
            tokenizer: HierarchicalAssemblyTokenizer instance
            node_feature_dim: Dimension of node embeddings
        """
        self.graph_list = graph_list
        self.tokenizer = tokenizer
        self.node_feature_dim = node_feature_dim

        # Initialize embedding layers for each feature type
        num_feature_type = 5
        vocab_sizes = tokenizer.get_vocab_sizes()
        self.node_type_embedding = nn.Embedding(vocab_sizes['node_type'], node_feature_dim // num_feature_type)
        self.operation_embedding = nn.Embedding(vocab_sizes['operation'], node_feature_dim // num_feature_type)
        self.register_embedding = nn.Embedding(vocab_sizes['register'], node_feature_dim // num_feature_type)
        self.memory_pattern_embedding = nn.Embedding(vocab_sizes['memory_pattern'], node_feature_dim // num_feature_type)
        self.immediate_embedding = nn.Embedding(vocab_sizes['immediate'], node_feature_dim // num_feature_type)

        # Process graphs into PyTorch Geometric Data objects
        self.data_list = []
        for graph in self.graph_list:
            # process graph info
            self.data_list.append(self._process_graph(graph['graph_input'],graph['name']))

    def _process_graph(self, graph, hash_name):
        """Convert a NetworkX graph to a PyTorch Geometric Data object"""
        # Create a node ID mapping for consecutive IDs
        node_mapping = {node: i for i, node in enumerate(graph.nodes())}

        # Get node features
        node_features_dict = self.tokenizer.encode_graph(graph)

        # Convert to tensor-friendly format
        x = torch.zeros((len(graph.nodes()), 7), dtype=torch.long)
        for node_id, features in node_features_dict.items():
            x[node_mapping[node_id]] = torch.tensor(features)

        # Create edge index
        edge_list = list(graph.edges())
        if not edge_list:
            # Handle case with no edges
            edge_index = torch.zeros((2, 0), dtype=torch.long)
        else:
            edge_index = self.optimize_edge_index(edge_list, node_mapping)

        # Endcode Hash name
        encoded_hash = self.tokenizer.encode_hash(hash_name)
        encoded_hash_tensor = torch.tensor(encoded_hash, dtype=torch.long)

        # Get embeddings before creating Data object
        x = self.get_embeddings(x)

        # Create Data object
        data = Data(x=x, edge_index=edge_index, hash_encoded=encoded_hash_tensor)
        return data

    def optimize_edge_index(self, edge_list, node_mapping):
        """
        Optimizes the creation of edge_index tensor for graph representation.
        Args:
            edge_list (list of tuples): List of edge tuples (src, tgt).
            node_mapping (dict): Dictionary mapping node IDs to integer indices.
        Returns:
            torch.Tensor: Optimized edge_index tensor.
        """
        if not edge_list:
            return torch.zeros((2, 0), dtype=torch.long)

        num_edges = len(edge_list)
        src_indices = torch.empty(num_edges, dtype=torch.long)
        tgt_indices = torch.empty(num_edges, dtype=torch.long)

        for i, (src, tgt) in enumerate(edge_list):
            src_indices[i] = node_mapping[src]
            tgt_indices[i] = node_mapping[tgt]

        return torch.stack((src_indices, tgt_indices), dim=0).contiguous()

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, idx):
        return self.data_list[idx]

    def get_embeddings(self, x):
        """Transform tokenized features into embeddings"""
        # Split features into their components
        node_type_idx = x[:, 0]
        operation_idx = x[:, 1]
        register_idx1 = x[:, 2]
        register_idx2 = x[:, 3]
        register_idx3 = x[:, 4]
        memory_pattern_idx = x[:, 5]
        immediate_idx = x[:, 6]

        # Get embeddings for each component
        node_type_emb = self.node_type_embedding(node_type_idx)
        operation_emb = self.operation_embedding(operation_idx)

        # Combine register embeddings (average them)
        register_emb = (self.register_embedding(register_idx1) +
                        self.register_embedding(register_idx2) +
                        self.register_embedding(register_idx3)) / 3

        memory_pattern_emb = self.memory_pattern_embedding(memory_pattern_idx)
        immediate_emb = self.immediate_embedding(immediate_idx)

        # Concatenate all embeddings
        return torch.cat([
            node_type_emb,
            operation_emb,
            register_emb,
            memory_pattern_emb,
            immediate_emb
        ], dim=1)

"""
Graph Attention Network for assembly code analysis
Args:
    dataset: AssemblyGraphDataset instance
    hidden_dim: Hidden dimension of GAT layers
    output_dim: Output dimension of node embeddings
    heads: Number of attention heads
    dropout: Dropout rate
"""
class AssemblyGAT(nn.Module):
    def __init__(self, node_feature_dim, hidden_dim=64, output_dim=453, heads=8, dropout=0.6, hash_dim=512):
        super(AssemblyGAT, self).__init__()
        self.conv1 = GATConv(node_feature_dim, hidden_dim, heads=heads, dropout=dropout)
        self.conv2 = GATConv(hidden_dim * heads, output_dim, heads=1, concat=False, dropout=dropout)
        self.dropout = dropout
        self.hash_linear = nn.Linear(hash_dim, node_feature_dim) # Linear layer for hash encoding, adjusted dim
        self.hash_dim = hash_dim

    def forward(self, x, edge_index, hash_encoded, batch):
        """Forward pass through the GAT model"""
        # First GAT layer with activation and dropout
        x = F.elu(self.conv1(x, edge_index))
        x = F.dropout(x, p=self.dropout, training=self.training)

        # Second GAT layer
        x = self.conv2(x, edge_index)
        x = global_mean_pool(x, batch)
        return x

# Function to parse digraph string (as in previous example)
def parse_digraph_string(digraph_str):
    """Parse the digraph string to create a NetworkX DiGraph"""
    # Simple parser for the provided format
    G = nx.DiGraph()

    # Regular expressions for node and edge definitions
    node_pattern = re.compile(r'"([^"]+)"\s*\[label\s*=\s*"([^"]+)"\]')
    edge_pattern = re.compile(r'"([^"]+)"\s*->\s*"([^"]+)"')

    # Extract nodes and edges
    for line in digraph_str.strip().split('\n'):
        line = line.strip()

        # Skip the Digraph G { and } lines
        if line == 'Digraph G {' or line == '}':
            continue

        # Parse node definitions
        node_match = node_pattern.search(line)
        if node_match:
            node_id = node_match.group(1)
            label = node_match.group(2)
            G.add_node(node_id, label=label)
            continue

        # Parse edge definitions
        edge_match = edge_pattern.search(line)
        if edge_match:
            source = edge_match.group(1)
            target = edge_match.group(2)
            G.add_edge(source, target)
    return G

In [ ]:
# Create empty list to store dataframes
# init parameters to loop the file loading by batch
files_meta_list = set(df_meta_train['name'].astype(str))
random_seed = 42

def split_dataloader(dataset, list_of_hash, batch_size, validation_split=0.5, shuffle=True):
    random.seed(random_seed) #set random seed for random module.
    torch.manual_seed(random_seed) #set random seed for torch.

    validation_size = int(len(dataset) * validation_split)
    train_size = len(dataset) - validation_size

    indices = list(range(len(dataset)))
    if shuffle:
        random.shuffle(indices)

    train_indices = indices[:train_size]
    validation_indices = indices[train_size:]

    train_dataset = Subset(dataset, train_indices)
    validation_dataset = Subset(dataset, validation_indices)

    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False) # Shuffle is handled by the initial indices shuffling.
    validation_dataloader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)

    train_hash = [list_of_hash[i] for i in train_indices]
    val_hash = [list_of_hash[i] for i in validation_indices]

    return train_dataloader, validation_dataloader, train_hash, val_hash

def process_batch(file_batch):
    """
    Processes a batch of files.
    Args:
        file_batch (list): A list of file paths.
    """
    graph_list_curr = []
    file_with_err = 0

    for full_path_file in file_batch:
        try:
            with open(full_path_file, 'r') as f:
                hash_file = full_path_file.split('.jso')[0]
                hash_file = hash_file.split(split_char)[-1]

                #f = open(full_path_file, 'r')
                digraph_str = f.read()

                # test if file content has no error tag
                if 'ERROR' in digraph_str:
                    file_with_err += 1
                    continue

                G = parse_digraph_string(digraph_str)
                graph_list_curr.append({
                    'name': hash_file,
                    'graph_input' : G
                })
                f.close()

        except FileNotFoundError:
            print(f"Error: File not found: {full_path_file}")
        except Exception as e:
            print(f"Error processing {full_path_file}: {e}")

    return graph_list_curr

def get_metadata_from_hash(hash_list):
    hash_meta_values = []
    for hash_name in hash_list:
        df_y_values = df_meta_train[df_meta_train['name'] == hash_name].copy()
        df_y_values.drop(columns=['name'], inplace=True)
        if len(df_y_values) > 0 :
             hash_meta_values.append(df_y_values.values[0]) # Get the first row's values
        else:
            print(f"{hash_name} not found in metadata")
    return hash_meta_values

def train_model(train_dataloader, train_hash, batch_size, epochs):
    if os.path.exists(model_save_file):
      if torch.cuda.is_available():
          model.load_state_dict(torch.load(model_save_file, weights_only=True))
      else:
          model.load_state_dict(torch.load(model_save_file, weights_only=True, map_location=torch.device('cpu')))

    model.train()
    for epoch in range(epochs):
        train_loss = 0.0
        count_batch = 0

        for batch in train_dataloader:
            optimizer.zero_grad()  # Zero gradients

            # get the different components of the dataset
            x = batch.x.to(device)
            edge_index = batch.edge_index.to(device)
            hash_encoded = batch.hash_encoded.to(device)
            #print(f"batch: {count_batch}  == x: {x.shape} edge_index: {edge_index.shape} ")
            list_hash_batch = train_hash[count_batch*batch_size:(count_batch+1)*batch_size]
            y_label = get_metadata_from_hash(list_hash_batch)
            y_label = np.array(y_label)
            y_label = torch.tensor(y_label, dtype=torch.float32).to(device)

            bincount_list = torch.bincount(batch.batch).tolist()
            if any(bincount_list): #checks if any values are not zero.
                batch_p = torch.cat([torch.full((num_nodes_graph_i,), i, device=device) for i, num_nodes_graph_i in enumerate(bincount_list)])
                batch_p = batch_p.to(device)
            else:
                # Handle the case where the list is empty. For example, create an empty tensor or raise an error.
                print("Warning: bincount_list is empty. Creating an empty tensor.")
                batch_p = torch.empty(0, dtype=torch.long, device=device) # create empty tensor.

            # Forward pass
            output = model(x, edge_index, hash_encoded, batch_p)
            count_batch += 1

            if output.shape != y_label.shape:
                print(f"Warning: output shape {output.shape} and y_label shape {y_label.shape} do not match.")
                break

            loss = F.binary_cross_entropy_with_logits(output, y_label.float())
            train_loss += loss.item()

            loss.backward(retain_graph=True)
            optimizer.step()

            if torch.cuda.is_available():
                torch.cuda.empty_cache()

        avg_train_loss = train_loss / len(train_dataloader)
        #print(f"Epoch {epoch+1}/{epochs}, Train Loss: {avg_train_loss:.4f}")

    # save model trained
    torch.save(model.state_dict(), model_save_file)

def validate_model_perf(validation_dataloader, val_hash, batch_size):
    if os.path.exists(model_save_file):
        model.load_state_dict(torch.load(model_save_file, weights_only=True))

    model.eval() #set the model to evaluation mode.
    all_true_label = []
    all_pred_label = []
    hash_v = []

    count_batch = 0
    with torch.no_grad():
        for batch in validation_dataloader:
            x = batch.x.to(device)
            edge_index = batch.edge_index.to(device)
            hash_encoded = batch.hash_encoded.to(device)

            list_hash_batch = val_hash[count_batch*batch_size:(count_batch+1)*batch_size]
            y_label = get_metadata_from_hash(list_hash_batch)
            y_label = np.array(y_label)
            y_label = torch.tensor(y_label, dtype=torch.float32).to(device)

            bincount_list = torch.bincount(batch.batch).tolist()
            if any(bincount_list): #checks if any values are not zero.
                batch_p = torch.cat([torch.full((num_nodes_graph_i,), i, device=device) for i, num_nodes_graph_i in enumerate(bincount_list)])
                batch_p = batch_p.to(device)
            else:
                # Handle the case where the list is empty. For example, create an empty tensor or raise an error.
                print("Warning: bincount_list is empty. Creating an empty tensor.")
                batch_p = torch.empty(0, dtype=torch.long, device=device) # create empty tensor.

            # Forward pass
            output = model(x, edge_index, hash_encoded, batch_p)
            count_batch += 1

            if output.shape != y_label.shape:
                print(f"Warning: output shape {output.shape} and y_label shape {y_label.shape} do not match.")
                break

            predicted_labels_prob = torch.sigmoid(output)
            prediction_labels = (predicted_labels_prob > 0.5).int()
            all_true_label.append(y_label.cpu())
            all_pred_label.append(prediction_labels.cpu())
            #hash_v.append(list_hash_batch)

    true_labels_tensor = torch.cat(all_true_label, dim=0) # Concatenate along dimension 0
    pred_labels_tensor = torch.cat(all_pred_label, dim=0) # Concatenate along dimension 0

    return pred_labels_tensor, true_labels_tensor

def run_files_in_batches(full_file_paths, batch_files_size=50, mode='train'):

    print(f"There are {len(full_file_paths)} files for {mode}")
    num_files = len(full_file_paths)
    nb_batch = (num_files // batch_files_size) +1

    hash_v = []
    hash_t = []
    all_true_label = []
    all_pred_label = []

    all_counts = {
        'node_types': [],
        'operations': [],
        'registers': [],
        'memory_patterns': [],
        'immediates': [],
        'unique_hashes': [],
    }
    print(f"**** Tokenization Processing")
    for i in range(0, num_files, batch_files_size):
        batch_files = full_file_paths[i:i + batch_files_size]

        #print(f"Tokenization Processing batch {i // batch_files_size + 1}/{nb_batch}: {len(batch_files)} files")
        batch_graph_list = process_batch(batch_files)
        if len(batch_graph_list) == 0:
            print("graph list = 0")
            break

        tokens = tokenizer.get_tokens_and_counts(batch_graph_list) # you will need to implement this
        all_counts.update(tokens)

    # Create a tokenizer from token counts
    tokenizer.fit_from_counts(all_counts)

    print(f"**** Dataset Processing & Training")
    for i in range(0, num_files, batch_files_size):
        batch_files = full_file_paths[i:i + batch_files_size]
        print(f"Dataset Processing batch {i // batch_files_size + 1}/{nb_batch}: {len(batch_files)} files")

        batch_graph_list = process_batch(batch_files)
        if len(batch_graph_list) == 0:
            print("graph list = 0")
            break

        dataset = AssemblyGraphDataset(batch_graph_list, tokenizer, node_feature_dim=num_feature_dim)
        curr_dataloader = DataLoader(dataset, batch_size=batch_data_size, shuffle=False)

        list_of_hash = [graph['name'] for graph in batch_graph_list]

        if mode == 'train':
            train_model(curr_dataloader, list_of_hash, batch_data_size, epochs=3)
            hash_t.append(list_of_hash)
        else:
            pred_labels, true_labels = validate_model_perf(curr_dataloader, list_of_hash, batch_data_size)
            all_true_label.append(true_labels.cpu())
            all_pred_label.append(pred_labels.cpu())
            hash_v.append(list_of_hash)

        if torch.cuda.is_available():
            torch.cuda.empty_cache()

    # save all labels as tensor to return if mode= 'validation'
    if mode == 'validation':
        all_true_label_tensor = torch.cat(all_true_label).cpu().numpy()
        all_pred_label_tensor = torch.cat(all_pred_label).cpu().numpy()

        return all_pred_label_tensor, all_true_label_tensor, hash_v
    elif mode == 'train':
        return hash_t

# MODE TEST ONLY

Generate prediction based on the requested list of hash (no true labels available)


In [ ]:
#test_init_file = r"test_set_to_predict-JC.csv"
#filename_test_results = r"test_prediction.csv"
df_test_init = pd.read_csv(test_init_file, sep=",")
col_to_predict = df_test_init.columns
print(df_test_init.shape)
df_test_init.head()

In [ ]:
print("test to be done later, when full training is completed")

batch_data_size = 2
batch_files_size = 2
num_feature_dim = 15

tokenizer = HierarchicalAssemblyTokenizer()
model = AssemblyGAT(node_feature_dim=num_feature_dim, hidden_dim=64, output_dim=453)
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)

model.load_state_dict(torch.load(model_save_file, weights_only=True))
model.eval()

#init test
df_all_labels = df_test_init
list_test_hash = []
#all_pred_labels = pd.DataFrame(columns=col_to_predict)

with torch.no_grad():
    # pour tous les hash du fichiers test
    print(f"**** Dataset Processing for Test")
    count_batch = 0  
    for count_test in range(0, len(df_test_init), batch_files_size):
      
        hash_names = df_test_init.iloc[count_test:count_test + batch_files_size]['name'].to_list()
        print(f"Batch {count_batch} : files {count_test} and next {batch_files_size}")
        full_file_test_name = [os.path.join(test_path_dir, file+'.json') for file in hash_names]
        #print(full_file_test_name)

        batch_graph_list = process_batch(full_file_test_name)
        #print("2 :",len(batch_graph_list))
        tokenizer.fit(batch_graph_list)

        test_dataset = AssemblyGraphDataset(batch_graph_list, tokenizer,  node_feature_dim=num_feature_dim)
        test_dataloader = DataLoader(test_dataset, batch_size=batch_data_size, shuffle=False)
        
        # loop on the batch from the dataloader
        for batch in test_dataloader:
            batch_hash_list = hash_names
            x = batch.x.to(device)
            edge_index = batch.edge_index.to(device)
            hash_encoded = batch.hash_encoded.to(device)

            # Create batch vector for global pooling
            num_graphs = batch.num_graphs #obtain the number of graphs in the batch
            #print("3:", num_graphs)
            num_nodes_per_graph = torch.bincount(batch.batch).tolist() # get number of nodes per graph.
            if any(num_nodes_per_graph): #checks if any values are not zero.
                batch_p = torch.cat([torch.full((num_nodes_graph_i,), i, device=device) for i, num_nodes_graph_i in enumerate(torch.bincount(batch.batch).tolist())])
                batch_p = batch_p.to(device)
            else:
                # Handle the case where the list is empty. For example, create an empty tensor or raise an error.
                print("Warning: bincount_list is empty. Creating an empty tensor.")
                batch_p = torch.empty(0, dtype=torch.long, device=device) # create empty tensor.

            output = model(x, edge_index, hash_encoded, batch_p)
            predicted_labels_prob = torch.sigmoid(output)
            prediction_labels = (predicted_labels_prob > 0.5).int()

            if len(prediction_labels) != len(batch_hash_list):
                print(f"*** error: len(prediction_labels)={len(prediction_labels)}, len(hash_names)={len(batch_hash_list)}")

            for i, hash_val in enumerate(batch_hash_list):
                row_index = df_all_labels[df_all_labels['name'] == hash_val].index
                #print(row_index,hash_val )
                if not row_index.empty:
                    label_list = prediction_labels[i].tolist()  # Convert tensor row to list
                    int_label_list = [int(val) for val in label_list]  # Convert to int

                    # Update the DataFrame
                    for j in range(len(int_label_list)):
                        df_all_labels.iloc[row_index, j+1] = int_label_list[j]
                else:
                    print(f"Warning: hash_val '{hash_val}' not found in df_all_labels.")

        count_batch += 1

    print(f"Test end: nb pred labels: {len(all_pred_labels)} ")
    df_all_labels.to_csv(filename_test_results, header=True ,index=False )

test to be done later, when full training is completed


NameError: name 'HierarchicalAssemblyTokenizer' is not defined

In [ ]:
print(df_all_labels.shape)
df_all_labels.head(10)

(11, 454)


,name,64-bit execution via heavens gate,64bits,PEB access,accept command line arguments,access the Windows event log,act as TCP client,allocate RW memory,allocate RWX memory,allocate memory,...,winzip,wise,worm,write and execute a file,write clipboard data,write file on Linux,write file on Windows,write pipe,xorcrypt,yoda
0,000fd5f03bb251ae0d81d8fd0d6aa86485639a17fe4a04...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0a4d467d954416807c8493f2fa73b0196e3ad8dc85ee5f...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0a4b7259c5a459697122a2c4e85d5499919fa6a9c91d66...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0a0c976cce2e9ce258a73bcb1d45a5c1a790fafc12c1ab...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,00da1ec8b4b2832056b13e77af850e2bbb22f4d36dd2ae...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0b709b99660397d1b4367ee14f404d7968811ddfea8f90...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
6,0b678fb57ca1d75961a48b58942a0cf9fccffae49225f6...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
7,0b99c9d49626e2bbdf1e3a87d55a6b454c994497534572...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
8,0b58a2bed7d9d44e29ad6988250456c49863a23adeb9b5...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
9,0b45ddc2ff31d966c0e1823e8d0c6b90fef9400d4a0ea2...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
